# 임베딩을 사용한 질문 답변

많은 사용 사례에서 인사이트가 담긴 답변으로 사용자 질문에 응답하기 위해 GPT-3가 필요합니다. 예를 들어 고객 지원 챗봇은 일반적인 질문에 대한 답변을 제공해야 할 수 있습니다. GPT 모델은 훈련을 통해 많은 일반 지식을 습득했지만, 보다 구체적인 정보가 담긴 대규모 라이브러리를 수집하고 사용해야 하는 경우가 많습니다.

이 노트북에서는 문서 임베딩 및 검색을 사용하여 텍스트 라이브러리를 참조로 사용하여 GPT-3가 질문에 답할 수 있도록 하는 방법을 보여드리겠습니다. 여기서는 2020 하계 올림픽에 관한 위키피디아 기사 데이터 세트를 사용합니다. 데이터 수집 프로세스는 [이 노트북](fine-tuned_qa/olympics-1-collect-data.ipynb)을 참조하세요.

-------

Many use cases require GPT-3 to respond to user questions with insightful answers. For example, a customer support chatbot may need to provide answers to common questions. The GPT models have picked up a lot of general knowledge in training, but we often need to ingest and use a large library of more specific information.

In this notebook we will demonstrate a method for enabling GPT-3 to answer questions using a library of text as a reference, by using document embeddings and retrieval. We'll be using a dataset of Wikipedia articles about the 2020 Summer Olympic Games. Please see [this notebook](fine-tuned_qa/olympics-1-collect-data.ipynb) to follow the data gathering process.

In [1]:
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken

COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDING_MODEL = "text-embedding-ada-002"

기본적으로 GPT-3는 2020년 올림픽에 대한 전문가는 아닙니다:

---

By default, GPT-3 isn't an expert on the 2020 Olympics:

In [2]:
prompt = "2020 하계올림픽 남자 높이뛰기에서 우승한 선수는 누구인가요?"
# prompt = "Who won the 2020 Summer Olympics men's high jump?"

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'2020 하계올림픽 남자 높이뛰기에서 우승한 선수는 독일의 마르크스 바이에르스트라이트입니다.'

마르셀로는 금메달리스트 수영 선수이지만 높이뛰기를 잘하지는 못합니다! GPT-3에 도움이 필요한 것 같습니다. 

가장 먼저 해결해야 할 문제는 모델이 "모르겠습니다"라고 말하지 않고 환각을 통해 대답한다는 것입니다. 이는 모델이 제공하는 답을 신뢰하기 어렵게 만들기 때문에 좋지 않습니다! 

-------

Marcelo is a gold medalist swimmer, and, we assume, not much of a high jumper! Evidently GPT-3 needs some assistance here. 

The first issue to tackle is that the model is hallucinating an answer rather than telling us "I don't know". This is bad because it makes it hard to trust the answer that the model gives us! 

# 0) 신속한 엔지니어링을 통한 환각 방지

프롬프트를 보다 명확하게 함으로써 이러한 환각 문제를 해결할 수 있습니다:

------

We can address this hallucination issue by being more explicit with our prompt:


In [3]:
prompt = """질문에 최대한 사실대로 답하고, 정답이 확실하지 않은 경우 "죄송합니다, 모릅니다"라고 답하세요.

질문: 2020 하계올림픽 남자 높이뛰기에서 우승한 선수는 누구인가요?
A:"""

# prompt = """Answer the question as truthfully as possible, and if you're unsure of the answer, say "Sorry, I don't know".

# Q: Who won the 2020 Summer Olympics men's high jump?
# A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'죄송합니다, 모릅니다.'

모델이 질문에 답할 수 있도록 프롬프트에 추가 컨텍스트 정보를 제공합니다. 필요한 전체 컨텍스트가 짧은 경우 프롬프트에 직접 포함할 수 있습니다. 예를 들어 Wikipedia에서 가져온 이 정보를 사용할 수 있습니다. 초기 프롬프트를 업데이트하여 모델이 제공된 텍스트를 명시적으로 사용하도록 지시합니다.

----

To help the model answer the question, we provide extra contextual information in the prompt. When the total required context is short, we can include it in the prompt directly. For example we can use this information taken from Wikipedia. We update the initial prompt to tell the model to explicitly make use of the provided text.

In [4]:
prompt = """제공된 텍스트를 사용하여 질문에 최대한 사실대로 답하고, 아래 텍스트에 답이 포함되어 있지 않은 경우 "모름"이라고 답하세요.

맥락:
2020 하계 올림픽 남자 높이뛰기 종목은 2021년 7월 30일부터 8월 1일 사이에 올림픽 스타디움에서 열렸습니다.
24개국에서 33명의 선수가 출전했으며, 총 출전 가능 인원은 유니버설 경기장 외에 추가로 선수를 출전시키기 위해 
를 사용하여 점수 또는 순위를 통해 예선에 진출한 32명 외에 추가로 출전할 수 있는 국가 수에 따라 총 출전 가능 인원이 결정되었습니다(2021년에는 유니버설 플레이스가 사용되지 않음).
이탈리아의 지안마르코 탐베리 선수와 카타르의 무타즈 에사 바르심 선수가 공동 우승자로 선정되었습니다.
두 선수는 2.37m를 통과해 동점을 기록했습니다. 탐베리와 바르심은 서로 다른 나라의 선수가 금메달을 나눠 갖는
올림픽 역사상 서로 다른 국가의 선수가 같은 메달을 공유하기로 합의한 드문 사례입니다. 
특히 바르심은 대회 관계자에게 "금메달 두 개를 주면 안 되나요?"라고 물은 것으로 알려졌습니다. 
'점프'. 동메달은 벨라루스의 막심 네다세카우가 차지했습니다. 이탈리아는 남자 높이뛰기에서 사상 첫 메달을 목에 걸었고 
벨라루스, 이탈리아와 카타르의 남자 높이뛰기 첫 금메달, 카타르의 남자 높이뛰기 3연속 메달
(모두 바르심의 기록입니다). 바르심은 높이뛰기에서 세 개의 메달을 획득한 두 번째 선수가 되었습니다.
(1984~1992)에 이어 두 번째로 세 개의 메달을 획득한 선수가 되었습니다.

질문: 2020 하계 올림픽 남자 높이뛰기에서 누가 우승했나요?
A:"""

# prompt = """Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "I don't know"

# Context:
# The men's high jump event at the 2020 Summer Olympics took place between 30 July and 1 August 2021 at the Olympic Stadium.
# 33 athletes from 24 nations competed; the total possible number depended on how many nations would use universality places 
# to enter athletes in addition to the 32 qualifying through mark or ranking (no universality places were used in 2021).
# Italian athlete Gianmarco Tamberi along with Qatari athlete Mutaz Essa Barshim emerged as joint winners of the event following
# a tie between both of them as they cleared 2.37m. Both Tamberi and Barshim agreed to share the gold medal in a rare instance
# where the athletes of different nations had agreed to share the same medal in the history of Olympics. 
# Barshim in particular was heard to ask a competition official "Can we have two golds?" in response to being offered a 
# 'jump off'. Maksim Nedasekau of Belarus took bronze. The medals were the first ever in the men's high jump for Italy and 
# Belarus, the first gold in the men's high jump for Italy and Qatar, and the third consecutive medal in the men's high jump
# for Qatar (all by Barshim). Barshim became only the second man to earn three medals in high jump, joining Patrik Sjöberg
# of Sweden (1984 to 1992).

# Q: Who won the 2020 Summer Olympics men's high jump?
# A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'이탈리아의 지안마르코 탐베리 선수와 카타르의 무타즈 에사 바르심 선수가 공동 우승자로 선정되었습니다.'

프롬프트에 추가 정보를 추가하는 것은 모델이 알아야 할 추가 콘텐츠의 데이터 세트가 단일 프롬프트에 들어갈 수 있을 정도로 작은 경우에만 작동합니다. 모델이 대규모 정보 중에서 관련성 있는 문맥 정보를 선택해야 할 때는 어떻게 해야 할까요?

**이 노트북의 나머지 부분에서는 문서 임베딩 및 검색을 사용하여 대량의 추가 문맥 정보로 GPT-3를 보강하는 방법을 보여드리겠습니다.** 이 방법은 먼저 쿼리와 관련된 정보를 검색한 다음 검색된 정보를 기반으로 질문에 맞는 답변을 작성하는 두 단계로 쿼리에 답합니다. 첫 번째 단계는 [임베딩 API](https://beta.openai.com/docs/guides/embeddings)를 사용하고, 두 번째 단계는 [완성 API](https://beta.openai.com/docs/guides/completion/introduction)를 사용합니다.
 
단계는 다음과 같습니다:
* 문맥 정보를 청크로 분할하여 전처리하고 각 청크에 대한 임베딩 벡터를 생성합니다.
* 쿼리를 받으면 문맥 청크와 동일한 벡터 공간에 쿼리를 임베딩하고 쿼리와 가장 유사한 문맥 임베딩을 찾습니다.
* 쿼리 프롬프트에 가장 관련성이 높은 컨텍스트 임베딩을 준비합니다.
* 가장 관련성이 높은 문맥과 함께 질문을 GPT에 제출하고 제공된 문맥 정보를 활용한 답변을 받습니다.

-----

Adding extra information into the prompt only works when the dataset of extra content that the model may need to know is small enough to fit in a single prompt. What do we do when we need the model to choose relevant contextual information from within a large body of information?

**In the remainder of this notebook, we will demonstrate a method for augmenting GPT-3 with a large body of additional contextual information by using document embeddings and retrieval.** This method answers queries in two steps: first it retrieves the information relevant to the query, then it writes an answer tailored to the question based on the retrieved information. The first step uses the [Embeddings API](https://beta.openai.com/docs/guides/embeddings), the second step uses the [Completions API](https://beta.openai.com/docs/guides/completion/introduction).
 
The steps are:
* Preprocess the contextual information by splitting it into chunks and create an embedding vector for each chunk.
* On receiving a query, embed the query in the same vector space as the context chunks and find the context embeddings which are most similar to the query.
* Prepend the most relevant context embeddings to the query prompt.
* Submit the question along with the most relevant context to GPT, and receive an answer which makes use of the provided contextual information.

# 1) 문서 라이브러리 전처리

문서 임베딩을 사용하여 문서 라이브러리에서 가장 관련성이 높은 부분을 가져와서 GPT-3에 제공하는 프롬프트에 삽입할 계획입니다. 따라서 문서 라이브러리를 개별적으로 검색 및 검색할 수 있는 컨텍스트의 "섹션"으로 분할해야 합니다. 

섹션은 질문에 답하기에 충분한 정보를 포함할 수 있을 만큼 충분히 커야 하지만, GPT-3 프롬프트에 하나 또는 여러 개를 넣을 수 있을 정도로 작아야 합니다. 일반적으로 대략 한 문단 정도의 텍스트가 적당하지만, 특정 사용 사례에 맞게 실험해 보아야 합니다. 이 예제에서는 Wikipedia 문서가 이미 의미적으로 관련된 헤더로 그룹화되어 있으므로 이를 사용하여 섹션을 정의하겠습니다. 이 전처리는 [이 노트북](fine-tuned_qa/olympics-1-collect-data.ipynb)에서 이미 완료되었으므로 그 결과를 로드하여 사용하겠습니다.

----------

We plan to use document embeddings to fetch the most relevant part of parts of our document library and insert them into the prompt that we provide to GPT-3. We therefore need to break up the document library into "sections" of context, which can be searched and retrieved separately. 

Sections should be large enough to contain enough information to answer a question; but small enough to fit one or several into the GPT-3 prompt. We find that approximately a paragraph of text is usually a good length, but you should experiment for your particular use case. In this example, Wikipedia articles are already grouped into semantically related headers, so we will use these to define our sections. This preprocessing has already been done in [this notebook](fine-tuned_qa/olympics-1-collect-data.ipynb), so we will load the results and use them.

In [5]:
# We have hosted the processed dataset, so you can download it directly without having to recreate it.
# This dataset has already been split into sections, one row for each section of the Wikipedia page.

df = pd.read_csv('./data/olympics_sections_text.csv')
# df = pd.read_csv('https://cdn.openai.com/API/examples/data/olympics_sections_text.csv')
df = df.set_index(["title", "heading"])
print(f"{len(df)} rows in the data.")
df.sample(5)

3964 rows in the data.


,,content,tokens
title,heading,,
Football at the 2020 Summer Olympics – Men's tournament,Summary,The men's football tournament at the 2020 Summ...,208
Water polo at the 2020 Summer Olympics – Men's team rosters,South Africa,The South Africa's squad was announced on 24 J...,52
Rowing at the 2020 Summer Olympics,Qualification,A total of 526 quota spots were available. Eac...,188
Senegal at the 2020 Summer Olympics,Wrestling,Senegal qualified one wrestler for the men's f...,52
Australia at the 2020 Summer Olympics,Athletics,Australian athletes further achieved the entry...,184


In [6]:
# We have hosted the processed dataset, so you can download it directly without having to recreate it.
# This dataset has already been split into sections, one row for each section of the Wikipedia page.

# df = pd.read_csv('./data/olympics_sections_text.csv')
df_ko = pd.read_csv('./data/olympics_sections_text_ko.csv')
# df = pd.read_csv('https://cdn.openai.com/API/examples/data/olympics_sections_text.csv')
df_ko = df_ko.set_index(["title", "heading"])
print(f"{len(df_ko)} rows in the data.")
df_ko.sample(5)

3964 rows in the data.


,,content,tokens
title,heading,,
2020년 하계 올림픽 오스트리아 선수단,도로,오스트리아는 UCI 세계 랭킹에서 전국 상위 50위(남성)와 개인 상위 100위(여...,56
2020년 하계 올림픽 골프,요약,일본 도쿄에서 열린 2020년 하계 올림픽 골프는 남녀 개인전 두 가지 종목으로 진...,123
2020년 하계 올림픽 혼합 복식 배드민턴,요약,2020년 하계 올림픽 혼합 복식 배드민턴 토너먼트가 7월 24일부터 30일까지 도...,47
2020년 하계 올림픽 조정 남자 에잇,배경,이것은 1896년 제1회 대회(악천후로 인해 모든 조정 경기가 취소됨)에 열리지 않...,103
2020년 하계 올림픽 불가리아 선수단,요약,불가리아는 2020년 도쿄 하계 올림픽에 참가했습니다. 당초 2020년 7월 24일...,164


각 섹션에 임베딩 벡터를 생성하여 문서 섹션을 사전 처리합니다. 임베딩은 텍스트가 의미적으로 얼마나 유사하거나 다른지 이해하는 데 도움이 되는 숫자 벡터입니다. 두 임베딩이 서로 가까울수록 그 내용이 더 유사합니다. 자세한 내용은 [OpenAI 임베딩 문서](https://beta.openai.com/docs/guides/embeddings)를 참조하세요.

이 인덱싱 단계는 오프라인에서 실행할 수 있으며, 데이터 세트의 인덱스를 미리 계산하여 나중에 각 콘텐츠를 검색할 수 있도록 한 번만 실행합니다. 이것은 작은 예시이므로 임베딩을 로컬에 저장하고 검색할 것입니다. 더 큰 데이터 세트가 있는 경우, [Pinecone](https://www.pinecone.io/) 또는 [Weaviate](https://github.com/semi-technologies/weaviate)와 같은 벡터 검색 엔진을 사용하여 검색을 강화하는 것을 고려해 보세요.

------------

We preprocess the document sections by creating an embedding vector for each section. An embedding is a vector of numbers that helps us understand how semantically similar or different the texts are. The closer two embeddings are to each other, the more similar are their contents. See the [documentation on OpenAI embeddings](https://beta.openai.com/docs/guides/embeddings) for more information.

This indexing stage can be executed offline and only runs once to precompute the indexes for the dataset so that each piece of content can be retrieved later. Since this is a small example, we will store and search the embeddings locally. If you have a larger dataset, consider using a vector search engine like [Pinecone](https://www.pinecone.io/), [Weaviate](https://github.com/semi-technologies/weaviate) or [Qdrant](https://qdrant.tech) to power the search.

In [7]:
import time
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def get_embedding(text: str, model: str=EMBEDDING_MODEL):# -> list[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame):# -> dict[tuple[str, str], list[float]]:
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.
    
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
    }

In [8]:
def load_embeddings(fname: str):# -> dict[tuple[str, str], list[float]]:
    """
    Read the document embeddings and their keys from a CSV.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """
    
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "title" and c != "heading"])
    return {
           (r.title, r.heading): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }

def load_embeddings1(fname: str):
    document_embeddings1 = np.load(fname, allow_pickle=True)
    document_embeddings2 = eval(str(document_embeddings1))
    return document_embeddings2

다시 한 번 말씀드리지만, 처음부터 다시 계산할 필요가 없도록 임베딩을 호스팅했습니다.

------

Again, we have hosted the embeddings for you so you don't have to re-calculate them from scratch.

In [9]:
document_embeddings = load_embeddings("./data/olympics_sections_document_embeddings.csv")

document_embeddings_ko = load_embeddings1("./data/olympics_sections_document_embeddings_ko.npy")

# ===== OR, uncomment the below line to recaculate the embeddings from scratch. ========

# document_embeddings_ko = compute_doc_embeddings(df)
# np.save("./data/olympics_sections_document_embeddings_ko.npy", document_embeddings_ko)

In [10]:
# An example embedding:
example_entry = list(document_embeddings.items())[0]
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

('2020 Summer Olympics', 'Summary') : [0.0037565305829048, -0.0061981128528714, -0.0087078781798481, -0.0071364338509738, -0.0025227521546185]... (1536 entries)


In [11]:
# An example embedding:
example_entry = list(document_embeddings_ko.items())[0]
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

('2020년 하계 올림픽', '요약') : [-0.0007347206119447947, -0.02737029641866684, 0.012973889708518982, -0.017294129356741905, -0.016912156715989113]... (1536 entries)


따라서 문서 라이브러리를 섹션으로 분할하고 각 청크를 나타내는 임베딩 벡터를 생성하여 인코딩했습니다. 이제 이 임베딩을 사용하여 사용자의 질문에 답하겠습니다.

----

So we have split our document library into sections, and encoded them by creating embedding vectors that represent each chunk. Next we will use these embeddings to answer our users' questions.

# 2) 질문 임베딩과 가장 유사한 문서 임베딩 찾기

질문 답변 시 사용자의 질문에 답변하기 위해 질문의 쿼리 임베딩을 계산하고 이를 사용하여 가장 유사한 문서 섹션을 찾습니다. 이것은 작은 예시이므로 임베딩을 로컬에 저장하고 검색합니다. 더 큰 데이터 세트가 있는 경우, [Pinecone](https://www.pinecone.io/) 또는 [Weaviate](https://github.com/semi-technologies/weaviate)와 같은 벡터 검색 엔진을 사용하여 검색을 강화하는 것을 고려해 보세요.

------

At the time of question-answering, to answer the user's query we compute the query embedding of the question and use it to find the most similar document sections. Since this is a small example, we store and search the embeddings locally. If you have a larger dataset, consider using a vector search engine like [Pinecone](https://www.pinecone.io/), [Weaviate](https://github.com/semi-technologies/weaviate) or [Qdrant](https://qdrant.tech) to power the search.

In [12]:
def vector_similarity(x: list[float], y: list[float]):# -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]):# -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [13]:
order_document_sections_by_query_similarity("Who won the men's high jump?", document_embeddings)[:5]

[(0.8848643084506062,
  ("Athletics at the 2020 Summer Olympics – Men's high jump", 'Summary')),
 (0.8633938355935514,
  ("Athletics at the 2020 Summer Olympics – Men's pole vault", 'Summary')),
 (0.8616397305838497,
  ("Athletics at the 2020 Summer Olympics – Men's long jump", 'Summary')),
 (0.8560523857031258,
  ("Athletics at the 2020 Summer Olympics – Men's triple jump", 'Summary')),
 (0.8469039130441246,
  ("Athletics at the 2020 Summer Olympics – Men's 110 metres hurdles",
   'Summary'))]

In [14]:
order_document_sections_by_query_similarity("Who won the men's high jump?", document_embeddings_ko)[:5]

[(0.8286446311289515, ('2020년 하계 올림픽 남자 장대높이뛰기 육상', '요약')),
 (0.82831505919221, ('2020년 하계 올림픽 육상 남자 높이뛰기', '요약')),
 (0.8081642654211549, ('2020년 하계 올림픽 카타르 선수단', '유도')),
 (0.8055039502460658, ('2020년 하계 올림픽 이란 선수단', '수영')),
 (0.8054196011556329, ('2020년 하계 올림픽 팀 점핑 승마 선수', '요약'))]

In [15]:
order_document_sections_by_query_similarity("남자 높이뛰기 우승은 누가 차지했을까요?", document_embeddings)[:5]

[(0.7956016404029377,
  ('South Korea at the 2020 Summer Olympics', 'Weightlifting')),
 (0.7936541193524953,
  ('South Korea at the 2020 Summer Olympics', 'Sport climbing')),
 (0.785071509176964, ('South Korea at the 2020 Summer Olympics', 'Taekwondo')),
 (0.7833206378896463, ('South Korea at the 2020 Summer Olympics', 'Fencing')),
 (0.7824215646099127, ('South Korea at the 2020 Summer Olympics', 'Road'))]

In [16]:
order_document_sections_by_query_similarity("남자 높이뛰기 우승은 누가 차지했을까요?", document_embeddings_ko)[:5]

[(0.8631184661114797, ('2020년 하계 올림픽 남자 장대높이뛰기 육상', '요약')),
 (0.8555872028407194, ('2020년 하계 올림픽 육상 남자 세단뛰기', '경쟁 형식')),
 (0.8510742281427929, ('2020년 하계 올림픽 남자 개인 양궁', '경쟁 형식')),
 (0.8507561232920778, ('2020년 하계 올림픽 육상 남자 멀리뛰기', '경쟁 형식')),
 (0.8491350731263664, ('2020년 하계 올림픽 남자 장대높이뛰기 육상', '자격'))]

In [17]:
order_document_sections_by_query_similarity("남자 높이뛰기 우승자는 누구일까요?", document_embeddings)[:5]

[(0.7935790951627928,
  ('South Korea at the 2020 Summer Olympics', 'Weightlifting')),
 (0.7881596031826634,
  ('South Korea at the 2020 Summer Olympics', 'Sport climbing')),
 (0.78351482754777, ('South Korea at the 2020 Summer Olympics', 'Taekwondo')),
 (0.7800376001118332, ('South Korea at the 2020 Summer Olympics', 'Fencing')),
 (0.7780477617782907,
  ('South Korea at the 2020 Winter Youth Olympics', 'Ice hockey'))]

In [18]:
order_document_sections_by_query_similarity("남자 높이뛰기 우승자는 누구일까요?", document_embeddings_ko)[:5]

[(0.8594850831427617, ('2020년 하계 올림픽 남자 장대높이뛰기 육상', '요약')),
 (0.8521748875501001, ('2020년 하계 올림픽 남자 개인 양궁', '경쟁 형식')),
 (0.8479371494525971, ('2020년 하계 올림픽 페루 선수단', '유도')),
 (0.847166054070742, ('2020년 하계 올림픽 남자 장대높이뛰기 육상', '자격')),
 (0.8467606835380376, ('2020년 하계 올림픽 남아프리카 선수단', '도로'))]

In [19]:
order_document_sections_by_query_similarity("남자 높이뛰기 우승은 누가 차지했나요?", document_embeddings)[:5]

[(0.7933649665878985,
  ('South Korea at the 2020 Summer Olympics', 'Weightlifting')),
 (0.7917448115310637,
  ('South Korea at the 2020 Summer Olympics', 'Sport climbing')),
 (0.7831762070756896,
  ('South Korea at the 2020 Summer Olympics', 'Taekwondo')),
 (0.7812067842986045, ('South Korea at the 2020 Summer Olympics', 'Fencing')),
 (0.7809279976321384, ('South Korea at the 2020 Summer Olympics', 'Road'))]

In [20]:
order_document_sections_by_query_similarity("남자 높이뛰기 우승은 누가 차지했나요?", document_embeddings_ko)[:5]

[(0.8602916217975068, ('2020년 하계 올림픽 남자 장대높이뛰기 육상', '요약')),
 (0.8539209305756412, ('2020년 하계 올림픽 육상 남자 세단뛰기', '경쟁 형식')),
 (0.8484875667423178, ('2020년 하계 올림픽 육상 남자 멀리뛰기', '경쟁 형식')),
 (0.8478048135236842, ('2020년 하계 올림픽 남자 개인 양궁', '경쟁 형식')),
 (0.8469847217342872, ('2020년 하계 올림픽 남아프리카 선수단', '도로'))]

In [21]:
order_document_sections_by_query_similarity("남자 높이뛰기 금메달", document_embeddings)[:5]

[(0.7635824688203928,
  ('South Korea at the 2020 Summer Olympics', 'Weightlifting')),
 (0.7634933252593934, ('South Korea at the 2020 Summer Olympics', 'Fencing')),
 (0.7632521282203887,
  ('South Korea at the 2020 Summer Olympics', 'Sport climbing')),
 (0.7594046542492103,
  ('South Korea at the 2020 Summer Olympics', 'Wrestling')),
 (0.757672244193949,
  ('South Korea at the 2020 Winter Youth Olympics', 'Ice hockey'))]

In [22]:
order_document_sections_by_query_similarity("남자 높이뛰기 금메달", document_embeddings_ko)[:5]

[(0.8571238754831306, ('2020년 하계 올림픽 남자 장대높이뛰기 육상', '요약')),
 (0.8522756320926594, ('2020년 하계 올림픽 수단 선수단', '유도')),
 (0.8435221692914268, ('2020년 하계 올림픽 육상 남자 높이뛰기', '요약')),
 (0.8394837244915722, ('2020년 하계 올림픽 이집트 선수단', '다이빙')),
 (0.8381237986269101, ('2020년 하계 올림픽 이집트 선수단', '양궁'))]

In [23]:
order_document_sections_by_query_similarity("남자 높이뛰기 금메달은 누가 차지했나요?", document_embeddings_ko)[:5]

[(0.8542594597855913, ('2020년 하계 올림픽 남자 장대높이뛰기 육상', '요약')),
 (0.8440834044838894, ('2020년 하계 올림픽 수단 선수단', '유도')),
 (0.8394626808988681, ('2020년 하계 올림픽 육상 남자 높이뛰기', '요약')),
 (0.8355090342397329, ('2020년 하계 올림픽 이집트 선수단', '다이빙')),
 (0.8327883143274583, ('2020년 하계 올림픽 여자 장대높이뛰기 육상', '요약'))]

In [24]:
order_document_sections_by_query_similarity("Who won the women's high jump?", document_embeddings)[:5]

[(0.8726165220223291,
  ("Athletics at the 2020 Summer Olympics – Women's long jump", 'Summary')),
 (0.8682196158313349,
  ("Athletics at the 2020 Summer Olympics – Women's high jump", 'Summary')),
 (0.8631915263706712,
  ("Athletics at the 2020 Summer Olympics – Women's pole vault", 'Summary')),
 (0.8609374262115411,
  ("Athletics at the 2020 Summer Olympics – Women's triple jump", 'Summary')),
 (0.8581515607285684,
  ("Athletics at the 2020 Summer Olympics – Women's 100 metres hurdles",
   'Summary'))]

In [25]:
order_document_sections_by_query_similarity("Who won the women's high jump?", document_embeddings_ko)[:5]

[(0.8289110716490353, ('2020년 하계 올림픽 여자 장대높이뛰기 육상', '요약')),
 (0.8237597651065901, ('2020년 하계 올림픽 육상 여자 멀리뛰기', '요약')),
 (0.8225830071055443, ('2020년 하계 올림픽 육상 여자 1500m', '요약')),
 (0.8190495825973025, ('2020년 하계 올림픽 베네수엘라 선수단', '여자 세단뛰기')),
 (0.8179918860669924, ('2020년 하계 올림픽 육상 여자 100m', '요약'))]

In [26]:
order_document_sections_by_query_similarity("여자 높이뛰기 우승은 누가 차지했을까요?", document_embeddings)[:5]

[(0.8045133435509194,
  ('South Korea at the 2020 Summer Olympics', 'Sport climbing')),
 (0.8043829668709866, ('South Korea at the 2020 Summer Olympics', 'Road')),
 (0.80005877834221,
  ('South Korea at the 2020 Summer Olympics', 'Weightlifting')),
 (0.7873142663762338, ('South Korea at the 2020 Summer Olympics', 'Boxing')),
 (0.7865130863113923,
  ('South Korea at the 2020 Summer Olympics', 'Taekwondo'))]

In [27]:
order_document_sections_by_query_similarity("여자 높이뛰기 우승은 누가 차지했을까요?", document_embeddings_ko)[:5]

[(0.8668776419160145, ('2020년 하계 올림픽 육상 여자 높이뛰기', '요약')),
 (0.8582323551653552, ('2020년 하계 올림픽 육상 여자 높이뛰기', '자격')),
 (0.8572834252398696, ('2020년 하계 올림픽 여자 장대높이뛰기 육상', '요약')),
 (0.8512330500592005, ('2020년 하계 올림픽 여자 장대높이뛰기 육상', '자격')),
 (0.8512006732125851, ('2020년 하계 올림픽 몽골 선수단', '양궁'))]

각 질문과 가장 관련성이 높은 문서 섹션에는 남자 및 여자 높이뛰기 경기 요약이 포함되어 있으며, 이는 우리가 예상한 것과 정확히 일치합니다.

--------

We can see that the most relevant document sections for each question include the summaries for the Men's and Women's high jump competitions - which is exactly what we would expect.

# 3) 가장 관련성이 높은 문서 섹션을 쿼리 프롬프트에 추가하기

가장 관련성이 높은 문맥을 계산한 후에는 제공된 쿼리에 이를 추가하여 프롬프트를 구성합니다. 쿼리 구분 기호를 사용하면 모델이 개별 텍스트를 구분하는 데 도움이 됩니다.

---------

Once we've calculated the most relevant pieces of context, we construct a prompt by simply prepending them to the supplied query. It is helpful to use a query separator to help the model distinguish between separate pieces of text.

In [28]:
MAX_SECTION_LEN = 500
SEPARATOR = "\n* "
ENCODING = "gpt2"  # encoding for text-davinci-003

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

f"Context separator contains {separator_len} tokens"

'Context separator contains 3 tokens'

In [29]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame):# -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"



def construct_prompt_ko(question: str, context_embeddings: dict, df: pd.DataFrame):# -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """제공된 문맥을 사용하여 질문에 최대한 사실대로 답하고, 아래 텍스트에 답이 포함되어 있지 않은 경우 "모름"이라고 답합니다.\n\n문맥:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

In [30]:
prompt = construct_prompt(
    "Who won the 2020 Summer Olympics men's high jump?",
    document_embeddings,
    df
)

print("===\n", prompt)

Selected 2 document sections:
("Athletics at the 2020 Summer Olympics – Men's high jump", 'Summary')
("Athletics at the 2020 Summer Olympics – Men's long jump", 'Summary')
===
 Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."

Context:

* The men's high jump event at the 2020 Summer Olympics took place between 30 July and 1 August 2021 at the Olympic Stadium. 33 athletes from 24 nations competed; the total possible number depended on how many nations would use universality places to enter athletes in addition to the 32 qualifying through mark or ranking (no universality places were used in 2021). Italian athlete Gianmarco Tamberi along with Qatari athlete Mutaz Essa Barshim emerged as joint winners of the event following a tie between both of them as they cleared 2.37m. Both Tamberi and Barshim agreed to share the gold medal in a rare instance where the athletes of different nations h

In [31]:
prompt = construct_prompt(
    "Who won the 2020 Summer Olympics men's high jump?",
    document_embeddings_ko,
    df_ko
)

print("===\n", prompt)

Selected 2 document sections:
('2020년 하계 올림픽 육상 남자 높이뛰기', '요약')
('2020년 하계 올림픽 남자 장대높이뛰기 육상', '요약')
===
 Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."

Context:

* 2020년 하계 올림픽 남자 높이뛰기 경기가 2021년 7월 30일부터 8월 1일까지 올림픽 스타디움에서 열렸습니다. 24개국에서 온 33명의 선수들이 경쟁했습니다. 가능한 총 수는 마크 또는 순위를 통해 자격을 갖춘 32개 국가 외에 얼마나 많은 국가가 보편성 위치를 사용하여 선수를 입력하는지에 따라 달라집니다(2021년에는 보편성 위치가 사용되지 않음). 이탈리아 선수 지안마르코 탐베리(Gianmarco Tamberi)와 카타르 선수 무타즈 에사 바르심(Mutaz Essa Barshim)이 2.37m를 동점으로 통과하며 공동 우승자로 부상했습니다. Tamberi와 Barshim은 올림픽 역사상 서로 다른 국가의 선수들이 같은 메달을 공유하기로 합의한 드문 경우에 금메달을 공유하기로 합의했습니다. 특히 Barshim은 대회 관계자에게 "두 개의 금메달을 가질 수 있습니까? "라고 묻는 것을 들었습니다. '점프 오프'를 제안받은 것에 대한 응답으로. 벨로루시의 Maksim Nedasekau가 동메달을 차지했습니다. 메달은 이탈리아와 벨라루스의 남자 높이뛰기 사상 최초, 이탈리아와 카타르의 남자 높이뛰기 첫 금메달, 카타르의 남자 높이뛰기 연속 3연패 메달(모두 Barshim)입니다. Barshim은 높이뛰기에서 3개의 메달을 획득한 두 번째 남자가 되었으며, 스웨덴의 Patrik Sj철berg(1984-1992)에 이어졌습니다.
* 2020년 하계 올림픽 남자 장대높이뛰기 경기가 202

In [32]:
prompt = construct_prompt_ko(
    "남자 높이뛰기 금메달",
    document_embeddings_ko,
    df_ko
)

print("===\n", prompt)

Selected 4 document sections:
('2020년 하계 올림픽 남자 장대높이뛰기 육상', '요약')
('2020년 하계 올림픽 수단 선수단', '유도')
('2020년 하계 올림픽 육상 남자 높이뛰기', '요약')
('2020년 하계 올림픽 이집트 선수단', '다이빙')
===
 제공된 문맥을 사용하여 질문에 최대한 사실대로 답하고, 아래 텍스트에 답이 포함되어 있지 않은 경우 "모름"이라고 답합니다.

문맥:

* 2020년 하계 올림픽 남자 장대높이뛰기 경기가 2021년 7월 31일부터 8월 3일까지 일본 국립 경기장에서 열렸습니다. 18개국에서 온 29명의 선수들이 경쟁했습니다. 스웨덴의 Armand Duplantis가 금메달을, 미국의 Christopher Nilsen이 은메달을, 브라질의 Thiago Braz가 동메달을 차지했습니다. 이 대회에서 스웨덴의 첫 승리이자 1952년 이후 남자 장대높이뛰기에서 모든 색상의 첫 메달이었습니다. 2016년에 우승한 Braz는 장대높이뛰기에서 여러 메달을 획득한 9번째 선수가 되었습니다.
* 수단은 남자 경량급(73kg)의 모하메드 압달라술을 올림픽에 보내 달라는 노사정위원회와 국제유도연맹의 초청을 받았다.
* 2020년 하계 올림픽 남자 높이뛰기 경기가 2021년 7월 30일부터 8월 1일까지 올림픽 스타디움에서 열렸습니다. 24개국에서 온 33명의 선수들이 경쟁했습니다. 가능한 총 수는 마크 또는 순위를 통해 자격을 갖춘 32개 국가 외에 얼마나 많은 국가가 보편성 위치를 사용하여 선수를 입력하는지에 따라 달라집니다(2021년에는 보편성 위치가 사용되지 않음). 이탈리아 선수 지안마르코 탐베리(Gianmarco Tamberi)와 카타르 선수 무타즈 에사 바르심(Mutaz Essa Barshim)이 2.37m를 동점으로 통과하며 공동 우승자로 부상했습니다. Tamberi와 Barshim은 올림픽 역사상 서로 다른 국가의 선수들이 같은 메달을 공유하기로 합의한 드문 경우에 금메달을 공유하기로 합

이제 질문과 가장 관련이 있는 문서 섹션을 확보했습니다. 마지막 단계로 질문에 대한 답을 얻기 위해 이 모든 것을 종합해 보겠습니다.

--------

We have now obtained the document sections that are most relevant to the question. As a final step, let's put it all together to get an answer to the question.

# 4) 컨텍스트에 따라 사용자의 질문에 답변합니다.

이제 관련 컨텍스트를 검색하고 프롬프트를 구성했으므로 이제 완성 API를 사용하여 사용자의 쿼리에 답변할 수 있습니다.

--------

Now that we've retrieved the relevant context and constructed our prompt, we can finally use the Completions API to answer the user's query.

In [33]:
COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 300,
    "model": COMPLETIONS_MODEL,
}

In [34]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embeddings: dict[(str, str), np.array],
    show_prompt: bool = False
):# -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")


def answer_query_with_context_ko(
    query: str,
    df: pd.DataFrame,
    document_embeddings: dict[(str, str), np.array],
    show_prompt: bool = False
):# -> str:
    prompt = construct_prompt_ko(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

In [35]:
answer_query_with_context("Who won the 2020 Summer Olympics men's high jump?", df, document_embeddings)

Selected 2 document sections:
("Athletics at the 2020 Summer Olympics – Men's high jump", 'Summary')
("Athletics at the 2020 Summer Olympics – Men's long jump", 'Summary')


'Gianmarco Tamberi and Mutaz Essa Barshim emerged as joint winners of the event following a tie between both of them as they cleared 2.37m. Both Tamberi and Barshim agreed to share the gold medal.'

In [36]:
answer_query_with_context("Who won the 2020 Summer Olympics men's high jump?", df_ko, document_embeddings_ko)

Selected 2 document sections:
('2020년 하계 올림픽 육상 남자 높이뛰기', '요약')
('2020년 하계 올림픽 남자 장대높이뛰기 육상', '요약')


"Gianmarco Tamberi and Mutaz Essa Barshim won the 2020 Summer Olympics men's high jump, sharing the gold medal."

In [37]:
answer_query_with_context("남자 높이뛰기 우승은 누가 차지했을까요?", df_ko, document_embeddings_ko)

Selected 3 document sections:
('2020년 하계 올림픽 남자 장대높이뛰기 육상', '요약')
('2020년 하계 올림픽 육상 남자 세단뛰기', '경쟁 형식')
('2020년 하계 올림픽 남자 개인 양궁', '경쟁 형식')


'Armand Duplantis가 금메달을 차지했습니다.'

In [38]:
answer_query_with_context("남자 높이뛰기 금메달", df_ko, document_embeddings_ko)

Selected 4 document sections:
('2020년 하계 올림픽 남자 장대높이뛰기 육상', '요약')
('2020년 하계 올림픽 수단 선수단', '유도')
('2020년 하계 올림픽 육상 남자 높이뛰기', '요약')
('2020년 하계 올림픽 이집트 선수단', '다이빙')


'이탈리아 선수 지안마르코 탐베리(Gianmarco Tamberi)와 카타르 선수 무타즈 에사 바르심(Mutaz Essa Barshim)이 2.37m를 동점으로 통과하며 공동 우승자로 부상했습니다. Tamberi와 Barshim은 올림픽 역사상 서로 다른 국가의 선수들이 같은 메달을 공유하기로 합의한 드문 경우에 금메달을 공유'

임베딩과 완성 API를 결합하여 방대한 추가 지식 기반을 사용하여 질문에 답할 수 있는 질문 답변 모델을 만들었습니다. 또한 답을 모르는 경우에도 이해할 수 있습니다! 

이 예에서는 위키피디아 문서의 데이터 세트를 사용했지만 이 데이터 세트는 책, 기사, 문서, 서비스 매뉴얼 등으로 대체할 수 있습니다. **GPT-3로 무엇을 만들 수 있을지 기대됩니다!**

------------

By combining the Embeddings and Completions APIs, we have created a question-answering model which can answer questions using a large base of additional knowledge. It also understands when it doesn't know the answer! 

For this example we have used a dataset of Wikipedia articles, but that dataset could be replaced with books, articles, documentation, service manuals, or much much more. **We can't wait to see what you create with GPT-3!**

# 더 많은 예제 보기

이제 몇 가지 예제를 더 살펴봅시다.

-------------

Let's have some fun and try some more examples.

In [39]:
query = "Why was the 2020 Summer Olympics originally postponed?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 1 document sections:
('Concerns and controversies at the 2020 Summer Olympics', 'Summary')

Q: Why was the 2020 Summer Olympics originally postponed?
A: The 2020 Summer Olympics were originally postponed due to the COVID-19 pandemic.


In [40]:
query = "Why was the 2020 Summer Olympics originally postponed?"
answer = answer_query_with_context_ko(query, df_ko, document_embeddings_ko)

print(f"\nQ: {query}\nA: {answer}")

Selected 1 document sections:
('2020년 하계 올림픽 바베이도스 선수단', '요약')

Q: Why was the 2020 Summer Olympics originally postponed?
A: Due to the COVID-19 pandemic.


In [41]:
query = "2020 하계 올림픽이 원래 연기된 이유"
answer = answer_query_with_context_ko(query, df_ko, document_embeddings_ko)

print(f"\nQ: {query}\nA: {answer}")

Selected 7 document sections:
('2020년 하계 올림픽 적도 기니 선수단', '요약')
('2020년 하계 올림픽 방송인 목록', '요약')
('2020년 하계 올림픽 양궁', '요약')
('2020년 하계 올림픽 서핑', '요약')
('2020년 하계 올림픽 팔라우 선수단', '요약')
('2020년 하계 올림픽 나우루', '요약')
('2020년 하계 올림픽 수단 선수단', '요약')

Q: 2020 하계 올림픽이 원래 연기된 이유
A: COVID-19 대유행


In [42]:
query = "2020년 하계 올림픽은 왜 연기되었나요?"
answer = answer_query_with_context_ko(query, df_ko, document_embeddings_ko)

print(f"\nQ: {query}\nA: {answer}")

Selected 7 document sections:
('2020년 하계 올림픽 방송인 목록', '요약')
('2020년 하계 올림픽 예선 수영', '요약')
('2020년 하계 올림픽 양궁', '요약')
('2020년 하계 올림픽 적도 기니 선수단', '요약')
('2020년 하계 올림픽 예선 카누', '요약')
('2020년 하계 올림픽 팔라우 선수단', '요약')
('2020년 하계 올림픽 수단 선수단', '요약')

Q: 2020년 하계 올림픽은 왜 연기되었나요?
A: COVID-19 대유행으로 인해 2021년으로 연기되었습니다.


In [43]:
query = "In the 2020 Summer Olympics, how many gold medals did the country which won the most medals win?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 2 document sections:
('2020 Summer Olympics medal table', 'Summary')
('List of 2020 Summer Olympics medal winners', 'Summary')

Q: In the 2020 Summer Olympics, how many gold medals did the country which won the most medals win?
A: The United States won the most medals overall, with 113, and the most gold medals, with 39.


In [44]:
query = "In the 2020 Summer Olympics, how many gold medals did the country which won the most medals win?"
answer = answer_query_with_context_ko(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 2 document sections:
('2020 Summer Olympics medal table', 'Summary')
('List of 2020 Summer Olympics medal winners', 'Summary')

Q: In the 2020 Summer Olympics, how many gold medals did the country which won the most medals win?
A: The United States won the most medals overall, with 113, and the most gold medals, with 39.


In [45]:
query = "In the 2020 Summer Olympics, how many gold medals did the country which won the most medals win?"
answer = answer_query_with_context(query, df_ko, document_embeddings_ko)

print(f"\nQ: {query}\nA: {answer}")

Selected 1 document sections:
('2020년 하계 올림픽 메달 테이블', '요약')

Q: In the 2020 Summer Olympics, how many gold medals did the country which won the most medals win?
A: The United States won 39 gold medals.


In [46]:
query = "In the 2020 Summer Olympics, how many gold medals did the country which won the most medals win?"
answer = answer_query_with_context_ko(query, df_ko, document_embeddings_ko)

print(f"\nQ: {query}\nA: {answer}")

Selected 1 document sections:
('2020년 하계 올림픽 메달 테이블', '요약')

Q: In the 2020 Summer Olympics, how many gold medals did the country which won the most medals win?
A: 39개


In [47]:
query = "2020 하계 올림픽에서 가장 많은 메달을 획득한 국가는 몇 개의 금메달을 획득했나요?"
answer = answer_query_with_context_ko(query, df_ko, document_embeddings_ko)

print(f"\nQ: {query}\nA: {answer}")

Selected 1 document sections:
('2020년 하계 올림픽 미국 선수단', '요약')

Q: 2020 하계 올림픽에서 가장 많은 메달을 획득한 국가는 몇 개의 금메달을 획득했나요?
A: 한국이 39개의 금메달을 획득했습니다.


google 번역 오류로 한국으로 번역되었음.

#### 원본
The United States, represented by the United States Olympic & Paralympic Committee (USOPC), competed at the 2020 Summer Olympics in Tokyo. Originally scheduled to take place in the summer of 2020, the Games were postponed to July 23 to August 8, 2021, due to the COVID-19 pandemic. U.S. athletes have appeared in every Summer Olympics of the modern era, with the exception of the 1980 Summer Olympics in Moscow, which the U.S. boycotted. The opening ceremony flag-bearers for the United States were baseball player Eddy Alvarez and basketball player Sue Bird. Javelin thrower Kara Winger was the flag-bearer for the closing ceremony.The country finished the Games with 113 medals, the most amongst all nations: 39 gold, 41 silver, and 33 bronze. These individual totals were each the highest of the Games, after a final-day tally of three gold medals (women's basketball, women's omnium, and women's volleyball) surpassed China's total of 38 golds. This was the third consecutive Summer Olympics that the U.S. was the medal table leader. Overall, the medal total was slightly lower than five years prior in Rio de Janeiro, where the United States won 46 gold and 121 total medals.
As Los Angeles will be the host city of the 2028 Summer Olympics, the United States, along with France, which is hosting the 2024 Summer Olympics in Paris, marched in the opening ceremony just before the host nation Japan.

#### 번역본
미국 올림픽 패럴림픽 위원회(USOPC)가 대표하는 미국은 도쿄에서 열리는 2020년 하계 올림픽에 참가했습니다. 원래 2020년 여름에 열릴 예정이었던 올림픽은 COVID-19 대유행으로 인해 2021년 7월 23일부터 8월 8일까지로 연기되었습니다. 미국이 보이콧한 1980년 모스크바 하계올림픽을 제외하고 미국 선수들은 근대 하계올림픽 때마다 출전했다. 미국의 개막식 기수는 야구선수 에디 알바레즈와 농구선수 수 버드였다. 폐막식의 기수는 창던지기 카라 윙거(Kara Winger)였다. 한국은 금메달 39개, 은메달 41개, 동메달 33개로 세계 최다인 113개의 메달을 획득하며 대회를 마쳤다. 이 개인 총계는 마지막 날 3개의 금메달(여자 농구, 여자 옴니엄, 여자 배구)의 집계가 중국의 총 38개 금메달을 넘어선 후 각각 올림픽 최고 기록을 세웠습니다. 이것은 미국이 메달 테이블 리더였던 세 번째 연속 하계 올림픽이었습니다. 전반적으로 총 메달 수는 미국이 46개의 금메달과 121개의 메달을 획득한 5년 전 리우데자네이루보다 약간 낮았습니다. 로스엔젤레스가 2028년 하계올림픽 개최지가 됨에 따라 미국은 2024년 파리에서 하계올림픽을 개최하는 프랑스와 함께 개최국 일본에 앞서 개회식에 입장했다.

In [48]:
query = "가장 많은 금메달을 획득한 국가는 몇 개의 금메달을 획득했나요?"
answer = answer_query_with_context_ko(query, df_ko, document_embeddings_ko)

print(f"\nQ: {query}\nA: {answer}")

Selected 4 document sections:
('2020 동계 청소년 올림픽 메달 표', '메달 표')
('2020년 하계 올림픽 단체전 승마', '최종 순위(점프 후)')
('2020년 하계 올림픽 미국 선수단', '농구')
('2020년 하계 올림픽 육상 여자 4 × 100m 계주', '결정적인')

Q: 가장 많은 금메달을 획득한 국가는 몇 개의 금메달을 획득했나요?
A: 미국이 46개의 금메달을 획득했습니다.


In [49]:
query = "What was unusual about the men’s shot put competition?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 2 document sections:
("Athletics at the 2020 Summer Olympics – Men's shot put", 'Summary')
("Athletics at the 2020 Summer Olympics – Men's discus throw", 'Summary')

Q: What was unusual about the men’s shot put competition?
A: The same three competitors received the same medals in back-to-back editions of the same individual event.


In [50]:
query = "What was unusual about the men’s shot put competition?"
answer = answer_query_with_context_ko(query, df_ko, document_embeddings_ko)

print(f"\nQ: {query}\nA: {answer}")

Selected 2 document sections:
('2020년 하계 올림픽 사격', '경쟁 형식')
('2020년 하계 올림픽 육상 남자 멀리뛰기', '경쟁 형식')

Q: What was unusual about the men’s shot put competition?
A: Men's shot put competition was replaced with a mixed team event consisting of three male-only events: 50m rifle prone, 50m pistol, and double trap.


In [51]:
query = "남자 포환던지기 경기의 특이한 점은 무엇인가요?"
answer = answer_query_with_context_ko(query, df_ko, document_embeddings_ko)

print(f"\nQ: {query}\nA: {answer}")

Selected 3 document sections:
('2020년 하계 올림픽 남자 개인 양궁', '요약')
('2020년 하계 올림픽 수영 혼성 4 × 100m 혼계영', '레이스 규칙')
('2020년 하계 올림픽 남자 개인 양궁', '경쟁 형식')

Q: 남자 포환던지기 경기의 특이한 점은 무엇인가요?
A: 리커브 양궁 종목으로 세계 양궁이 승인한 70m 거리 및 규정에 따라 진행되며, 각 경기는 궁수 1인당 3발씩 최대 5세트로 구성되며, 녹아웃 경기는 2012년 도입된 세트제를 사용한다는 점입니다.


In [52]:
query = "In the 2020 Summer Olympics, how many silver medals did Italy win?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 2 document sections:
('Italy at the 2020 Summer Olympics', 'Summary')
('San Marino at the 2020 Summer Olympics', 'Summary')

Q: In the 2020 Summer Olympics, how many silver medals did Italy win?
A: 10 silver medals.


질의응답 모델은 답변이 엉뚱한 방향으로 흘러가는 경향이 적고, 무엇을 아는지 모르는지 더 잘 파악할 수 있습니다. 문맥에 정보가 포함되어 있지 않거나, 질문이 무의미한 경우, 또는 이론적으로는 답변이 가능하지만 GPT-3의 능력을 넘어서는 질문일 때 효과적입니다!

----------

Our Q&A model is less prone to hallucinating answers, and has a better sense of what it does or doesn't know. This works when the information isn't contained in the context; when the question is nonsensical; or when the question is theoretically answerable but beyond GPT-3's powers!

In [53]:
query = "What is the total number of medals won by France, multiplied by the number of Taekwondo medals given out to all countries?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 4 document sections:
('France at the 2020 Summer Olympics', 'Taekwondo')
('Taekwondo at the 2020 Summer Olympics – Qualification', 'Qualification summary')
('2020 Summer Olympics medal table', 'Medal count')
("Taekwondo at the 2020 Summer Olympics – Men's 80 kg", 'Competition format')

Q: What is the total number of medals won by France, multiplied by the number of Taekwondo medals given out to all countries?
A: I don't know.


In [54]:
query = "What is the total number of medals won by France, multiplied by the number of Taekwondo medals given out to all countries?"
answer = answer_query_with_context_ko(query, df_ko, document_embeddings_ko)

print(f"\nQ: {query}\nA: {answer}")

Selected 4 document sections:
('2020년 하계 올림픽 프랑스 선수단', '태권도')
('2020년 하계 올림픽 예선 태권도', '자격 요약')
('2020년 하계 올림픽 태국 선수단', '승마자')
('2020년 하계 올림픽 프랑스 선수단', '권투')

Q: What is the total number of medals won by France, multiplied by the number of Taekwondo medals given out to all countries?
A: 모름


In [55]:
query = "프랑스가 획득한 총 메달 수에 모든 국가에 수여된 태권도 메달 수를 곱한 값은 얼마입니까?"
answer = answer_query_with_context_ko(query, df_ko, document_embeddings_ko)

print(f"\nQ: {query}\nA: {answer}")

Selected 3 document sections:
('2020 동계 청소년 올림픽 메달 표', '메달 표')
('2020년 하계 올림픽 프랑스 선수단', '태권도')
('2020년 하계 올림픽 프랑스 선수단', '권투')

Q: 프랑스가 획득한 총 메달 수에 모든 국가에 수여된 태권도 메달 수를 곱한 값은 얼마입니까?
A: 모름


In [56]:
query = "What is the tallest mountain in the world?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 3 document sections:
("Ski mountaineering at the 2020 Winter Youth Olympics – Boys' individual", 'Summary')
("Sport climbing at the 2020 Summer Olympics – Men's combined", 'Route-setting')
("Ski mountaineering at the 2020 Winter Youth Olympics – Girls' individual", 'Summary')

Q: What is the tallest mountain in the world?
A: I don't know.


In [57]:
query = "What is the tallest mountain in the world?"
answer = answer_query_with_context_ko(query, df_ko, document_embeddings_ko)

print(f"\nQ: {query}\nA: {answer}")

Selected 1 document sections:
('2020년 하계 올림픽 육상 남자 높이뛰기', '요약')

Q: What is the tallest mountain in the world?
A: 모름


In [58]:
query = "세계에서 가장 높은 산은 무엇인가요?"
answer = answer_query_with_context_ko(query, df_ko, document_embeddings_ko)

print(f"\nQ: {query}\nA: {answer}")

Selected 6 document sections:
('2020년 하계 올림픽에서 세운 세계 및 올림픽 기록', '요약')
('2020년 하계 올림픽 육상', '수치 기록')
('2020년 하계 올림픽 뉴질랜드 선수단', '서핑')
('2020년 하계 올림픽 부룬디 선수단', '체육 실기')
('2020년 하계 올림픽 모로코 선수단', '서핑')
('2020년 하계 올림픽 혼성 단체 양궁', '경쟁 대진표')

Q: 세계에서 가장 높은 산은 무엇인가요?
A: 모름


In [59]:
query = "Who won the grimblesplatch competition at the 2020 Summer Olympic games?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 2 document sections:
("Gymnastics at the 2020 Summer Olympics – Women's trampoline", 'Summary')
('Equestrian at the 2020 Summer Olympics – Team jumping', 'Summary')

Q: Who won the grimblesplatch competition at the 2020 Summer Olympic games?
A: I don't know.


In [ ]:
query = "Who won the grimblesplatch competition at the 2020 Summer Olympic games?"
answer = answer_query_with_context_ko(query, df_ko, document_embeddings_ko)

print(f"\nQ: {query}\nA: {answer}")

In [ ]:
query = "2020 하계 올림픽 그림블스플래치 대회 우승자?"
answer = answer_query_with_context_ko(query, df_ko, document_embeddings_ko)

print(f"\nQ: {query}\nA: {answer}")